###🌐 Use Case Overview

➤ We are taking some content from a website (example: LangSmith docs)

➤ The goal is to scrape the webpage content, split it into smaller parts, convert it into vectors, and use it for Q&A with an LLM

➤ This is done using LangChain, OpenAI, and some additional tools


In [ ]:
# 🛠️ 1. Setup & Environment Variables
import os
from dotenv import load_dotenv
load_dotenv()

# os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAINAPI_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
# ✅ What's happening?
# •	You're loading secret keys (API keys) from a .env file.
# •	This lets your code talk to services like OpenAI and LangChain.

# 🌐 2. Load Website Data
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://python.langchain.com/v0.1/docs/langsmith/")
docs = loader.load()
# ✅ What's happening?
# •	You're loading the website data.
# •	This is a list of all the pages on the LangChain website.
# •	docs now contains the text from the page.

# 📄 3. Split the Webpage into Chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
documents
# [Document(metadata={'source': 'https://www.msn.com/en-in/money/news/petrol-diesel-fresh-prices-announced-check-rates-in-your-city-on-september-4/ar-AA1LPdK9?ocid=msedgntp&pc=DCTS&cvid=9efbc1097375464f9313f8964af0189c&ei=57', 'title': 'MSN', 'language': 'en-in'}, page_content='MSN')]

# ✅ What's happening?
# •	Large documents are hard for the AI to process.
# •	So, you're breaking them into smaller chunks (1000 characters with 200 characters overlap).
# •	This makes it easier to work with.

# 🤖 4. Convert Text Chunks into Vectors
# from langchain_openai import OpenAIEmbeddings
# embeddings = OpenAIEmbeddings()

from langchain_community.embeddings import OllamaEmbeddings
embeddings = OllamaEmbeddings(model="nomic-embed-text")
# C:\Users\sahus\AppData\Local\Temp\ipykernel_2812\1129778531.py:7: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
#   embeddings = OllamaEmbeddings(model="nomic-embed-text")

from langchain_community.vectorstores import FAISS
vectorstoredb = FAISS.from_documents(documents, embeddings)
vectorstoredb
# <langchain_community.vectorstores.faiss.FAISS at 0x2367b91dd30>
# ✅ What's happening?
# •	The text chunks are converted into embeddings (mathematical representations).
# •	These are stored in a vector database (FAISS), so you can search them later using questions.


# ❓ 5. Test a Search Query
query = "Key Factors Influencing Fuel Prices in India"
result = vectorstoredb.similarity_search(query)
result
# [Document(id='2af89cbd-4acb-4d5a-9338-ca1a6a0b9694', metadata={'source': 'https://www.msn.com/en-in/money/news/petrol-diesel-fresh-prices-announced-check-rates-in-your-city-on-september-4/ar-AA1LPdK9?ocid=msedgntp&pc=DCTS&cvid=9efbc1097375464f9313f8964af0189c&ei=57', 'title': 'MSN', 'language': 'en-in'}, page_content='MSN')]
# ✅ What's happening?
# •	You ask a natural language question.
# •	The system finds the most similar document chunk from the vector DB.

# 💬 6. Initialize the LLM (GPT-4o)
# from langchain_openai import ChatOpenAI
# llm = ChatOpenAI(model="gpt-4o")

from langchain_community.chat_models import ChatOllama
llm = ChatOllama(model="gemma3:1b")
print(llm)
# model='gemma3:1b'
# C:\Users\sahus\AppData\Local\Temp\ipykernel_2812\3204733895.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
#   llm = ChatOllama(model="gemma3:1b")

# ✅ What's happening?
# •	You’re telling LangChain to use OpenAI's GPT-4o as the brain to answer questions.

# 🔗 7. Create a Prompt Template and Document Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:
<context>
{context}
</context>
""")
document_chain = create_stuff_documents_chain(llm, prompt)
document_chain
# RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
#   context: RunnableLambda(format_docs)
# }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
# | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
# | ChatOllama(model='gemma3:1b')
# | StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])


# ✅ What's happening?
# •	You make a custom prompt telling GPT to only use the context provided (not guess).
# •	The document chain connects the prompt and the LLM

# 🔄 9. Turn the Vector DB into a Retriever
# Retriever is a interface when user asked question it will retrieve relevant document from vector db
# and pass to llm
retriever = vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain = create_retrieval_chain(retriever, document_chain)
retrieval_chain
# RunnableBinding(bound=RunnableAssign(mapper={
#   context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
#            | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002367B91DD30>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
# })
# | RunnableAssign(mapper={
#     answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
#               context: RunnableLambda(format_docs)
#             }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
#             | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
#             | ChatOllama(model='gemma3:1b')
#             | StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])
#   }), kwargs={}, config={'run_name': 'retrieval_chain'}, config_factories=[])

# ✅ What's happening?
# •	Now you're connecting everything:
# o	Retriever finds the most relevant text from the DB.
# o	Document chain sends it to the LLM.
# •	retrieval_chain is your final smart pipeline.

# 🧠 10. Ask a Question and Get Final Answer
response = retrieval_chain.invoke({"input": "Key Factors Behind Petrol and Diesel Rates"})
# ✅ What's happening?
# •	You ask a question.
# •	The system:
# 1.	Retrieves the most relevant documents.
# 2.	Passes them to the LLM with your question.
# 3.	Returns a smart answer.

# 🔍 11. Check the Answer & Context
response['answer']       # The final answer
response['context']      # The document(s) used to answer







In [ ]:
# 1️⃣ Load documents (PDFs, text, etc.)
# 2️⃣ Split into chunks ➡️ called document chunks
# 3️⃣ Convert to embeddings using OpenAI Embeddings
# 4️⃣ Store embeddings in Vector DB (like FAISS)
# 5️⃣ Query DB via Similarity Search ✅ (basic)
# 6️⃣ Create Document Chain to give context to LLM
# 7️⃣ Create Retriever ➡️ Automate fetching context
# 8️⃣ Combine Retriever + Document Chain ➡️ Retrieval Chain
# 9️⃣ Call Retrieval Chain with input ➡️ Get smart answers ✅

